In [ ]:
from grid.robot.aerial.airgen_drone import AirGenDrone 
airgen_drone__0 = AirGenDrone()

In [ ]:
airgen_drone_0.client.takeoffAsync().join()
airgen_drone_0.client.moveToZAsync(-25, 2).join()

In [ ]:
import airgen
import numpy as np
from grid.model.perception.detection.gdino import GroundingDINO

groundingdino = GroundingDINO()

# Define a detect function that captures an RGB image from the front camera and uses an object detector

def detect(drone, object_name="fire"):
    rgb_image, pose = drone.getImages("front_center", [airgen.ImageType.Scene])[0]
    boxes, phrases = groundingdino.detect_object(rgb_image, object_name)
    return boxes, phrases

# Let's search for the fire by rotating 360 degrees and trying to detect fire at each step
yaw_angles = np.linspace(0, 360, 30)

for yaw in yaw_angles:
    airgen_drone_0.client.rotateToYawAsync(yaw).join()
    boxes, phrases = detect(airgen_drone_0.client)
    if 'fire' in phrases:
        break

In [ ]:
# Let's try to use a different model and object category

from grid.model.perception.segmentation.clipseg import CLIPSeg
clipseg = CLIPSeg()

# Use CLIP features to try and segment out 'smoke'

def segment(drone_client, object_name="smoke"):
    rgb_image, pose = drone_client.getImages("front_center", [airgen.ImageType.Scene])[0]
    result = clipseg.segment_image(rgb_image, object_name)
    return result

segment(airgen_drone_0.client)

In [ ]:
# Let's evaluate the vision models with some weather variations to the scene
import time
airgen_drone_0.client.simEnableWeather(True)

for i in range(10):
    airgen_drone_0.client.simSetWeatherParameter(airgen.WeatherParameter.Rain, i / 10)
    airgen_drone_0.client.simSetWeatherParameter(airgen.WeatherParameter.Fog, i / 10)
    detect(airgen_drone_0.client)
    segment(airgen_drone_0.client)
    time.sleep(1)